In [1]:
# Подключение модулей
from keras.models import Sequential, Model
from keras.layers import Lambda, Input
from keras.backend import tf as ktf
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import cv2, numpy as np
import keras

In [2]:
#загрузили данные
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# Размер изображений
img_width, img_height = 28, 28
# Размер мини-выборки
batch_size = 100
# Кол-во изображений для обучения
nb_train_samples = 60000
# Кол-во изображений для теста
nb_test_samples = 10000

In [4]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = np.stack((x_train,)*3, axis=-1)
x_test = np.stack((x_test,)*3, axis=-1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [5]:
# Создание экземпляра модели сети MobileNet
mobile_net = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточная часть сети
mobile_net.trainable = True
trainable = False
for layer in mobile_net.layers:
  if layer.name == 'conv_dw_13':
    trainable = True
  layer.trainable = trainable

mobile_net.summary()

17235968/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________

In [6]:
# 3 channel images of arbitrary shape
inp = Input(shape=(None, None, 3))
out = Lambda(lambda image: ktf.image.resize(image, (128, 128)))(inp)

inputLayer = Model(inputs=inp, outputs=out, name="resizer")
inputLayer.summary()

Model: "resizer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
lambda (Lambda)              (None, 128, 128, 3)       0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Создание модели составной сети
model = Sequential()
# Добавляем слой для преобразования размера изображения
model.add(inputLayer)
# Добавляем сверточные слои
model.add(mobile_net)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizer (Functional)         (None, 128, 128, 3)       0         
_________________________________________________________________
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               4194560   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 7,425,994
Trainable params: 5,259,018
Non-trainable params: 2,166,976
______________________________________

In [8]:
# Компилируем составную сеть
epochs = 5
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
# Обучение составной сети
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = epochs, batch_size=batch_size)

Epoch 1/5
600/600 [==============================] - 78s 48ms/step - loss: 1.0619 - accuracy: 0.7276 - val_loss: 0.0595 - val_accuracy: 0.9813
Epoch 2/5
600/600 [==============================] - 25s 42ms/step - loss: 0.1105 - accuracy: 0.9656 - val_loss: 0.0413 - val_accuracy: 0.9868
Epoch 3/5
600/600 [==============================] - 25s 41ms/step - loss: 0.0656 - accuracy: 0.9795 - val_loss: 0.0355 - val_accuracy: 0.9889
Epoch 4/5
600/600 [==============================] - 27s 45ms/step - loss: 0.0484 - accuracy: 0.9842 - val_loss: 0.0316 - val_accuracy: 0.9902
Epoch 5/5
600/600 [==============================] - 27s 45ms/step - loss: 0.0357 - accuracy: 0.9885 - val_loss: 0.0293 - val_accuracy: 0.9906


In [10]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 99.06%
